In [ ]:
!pip install --upgrade --no-cache-dir gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0


In [ ]:
!gdown https://drive.google.com/u/1/uc?id=1t6byETrCpUIx2gZjEK6B4TyFWYHw0ijj&export=download&confirm=t

Downloading...
From: https://drive.google.com/u/1/uc?id=1t6byETrCpUIx2gZjEK6B4TyFWYHw0ijj
To: /content/new_dataset.zip
100% 144M/144M [00:01<00:00, 102MB/s]


In [ ]:
!unzip new_dataset.zip

Archive:  new_dataset.zip
   creating: new_dataset/
   creating: new_dataset/matching/
  inflating: new_dataset/matching/matching_3_(224x224).jpg  
  inflating: new_dataset/matching/matching_5_(224x224).jpg  
  inflating: new_dataset/matching/matching_6_(224x224).jpg  
  inflating: new_dataset/matching/matching_8_(224x224).jpg  
  inflating: new_dataset/matching/matching_9_(224x224).jpg  
  inflating: new_dataset/matching/matching_15_(224x224).jpg  
  inflating: new_dataset/matching/matching_16_(224x224).jpg  
  inflating: new_dataset/matching/matching_18_(224x224).jpg  
  inflating: new_dataset/matching/matching_19_(224x224).jpg  
  inflating: new_dataset/matching/matching_20_(224x224).jpg  
  inflating: new_dataset/matching/matching_21_(224x224).jpg  
  inflating: new_dataset/matching/matching_23_(224x224).jpg  
  inflating: new_dataset/matching/matching_24_(224x224).jpg  
  inflating: new_dataset/matching/matching_25_(224x224).jpg  
  inflating: new_dataset/matching/matching_26_(224

In [ ]:
dataset_location='./new_dataset'
batch_size = 32
n_epochs = 20
validation_split = 0.2
input_shape = (224, 224, 3)
img_size = (224, 224)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import matplotlib.pyplot as plt
import tensorflow_hub as hub


In [ ]:
from tensorflow.keras.applications import EfficientNetB7

In [ ]:
input_shape = (224, 224, 3)

#load the EfficientNetB7 model
model = EfficientNetB7(include_top=False, input_shape=input_shape)

#block the imported model from beein trained
for layer in model.layers:
    layer.trainable = False

model.summary()

Model: "efficientnetb7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling_1 (Rescaling)        (None, 224, 224, 3)  0           ['input_2[0][0]']                
                                                                                                  
 normalization_1 (Normalization  (None, 224, 224, 3)  7          ['rescaling_1[0][0]']            
 )                                                                                                
                                                                                     

In [ ]:
#add fully connected layers to classify nine classes
model = tf.keras.Sequential([
    model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 1)                 128455064 
                                                                 
 flatten_2 (Flatten)         (None, 1)                 0         
                                                                 
 dense_6 (Dense)             (None, 256)               512       
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                16448     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 128,472,089
Trainable params: 64,374,402

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        dataset_location,
        validation_split=validation_split,
        subset="training",
        seed=42,
        image_size=img_size,
        batch_size=batch_size)
    
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        dataset_location,
        validation_split=validation_split,
        subset="validation",
        seed=42,
        image_size=img_size,
        batch_size=batch_size)

Found 4178 files belonging to 2 classes.
Using 3343 files for training.
Found 4178 files belonging to 2 classes.
Using 835 files for validation.


In [ ]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential_1 (Sequential)   (None, 1)                 128455064 
                                                                 
 flatten_2 (Flatten)         (None, 1)                 0         
                                                                 
 dense_6 (Dense)             (None, 256)               512       
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 64)                16448     
                                                                 
 dense_8 (Dense)             (None, 1)                 65        
                                                                 
Total params: 128,472,089
Trainable params: 64,374,402

In [ ]:
history = model.fit(train_ds, epochs=n_epochs, validation_data=val_ds)

Epoch 1/20
105/105 [==============================] - ETA: 0s - loss: nan - accuracy: 0.5007

In [ ]:
#validate the model
loss_train, acc_train = model.evaluate(train_ds)
loss_val, acc_val = model.evaluate(val_ds)
    
print("Train accuracy: ", acc_train)
print("Validation accuracy: ", acc_val)

In [ ]:
# plot loss learning curves
plt.subplot(211)
plt.title('Cross-Entropy Loss', pad=-40)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()

# plot accuracy learning curves
plt.subplot(212)
plt.title('Accuracy', pad=-40)
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='test')
plt.legend()
plt.tight_layout()
plt.show()